In [1]:
#!/usr/bin/env python
# coding: utf-8

import numpy as np
import pandas as pd
import random
import math

from pandas.core.frame import DataFrame


class Node:
    def __init__(self):
        self.inEdgeList = []
        self.outEdgeList = []
        self.value = None
        self.delta = None
    
    def addInEdge(self, edge):
        self.inEdgeList.append(edge)
    
    def addOutEdge(self, edge):
        self.outEdgeList.append(edge)

        
class InputNode(Node):
    pass

class OutputNode(Node):
    pass

class HiddenNode(Node):
    pass

class Edge:
    def __init__(self, fromNode, toNode):
        self.fromNode = fromNode
        self.toNode = toNode
        self.weight = None


class Layer:
    def __init__(self):
        self.nodes = []
    
    def addNode(self, node):
        self.nodes.append(node)
    def removeNode(self, node):
        self.nodes.remove(node)

class InputLayer(Layer):
    pass

class OutputLayer(Layer):
    pass

class HiddenLayer(Layer):
    pass

class Utility:
    def logistic(node: Node) -> float:
        sumValue = 0.0
        for edge in node.inEdgeList:            
            sumValue += edge.weight * edge.fromNode.value
        ans = 1/ (1 + math.exp(-sumValue))
        return ans

        

class Graph:
    def __init__(self):
        self.inputLayer = InputLayer()
        self.outputLayer = OutputLayer()
        self.hiddenLayerList = []
        self.nodeList = []
        self.edgeList = []
        self.trainDf = None
        self.testDf = None
        self.maxAttribList = []
        self.minAttribList = []
        self.targetList = []
        self.learningRate = 0.1
    
    def createHiddenLayer(self):
        layer = HiddenLayer()        
        self.hiddenLayerList.append(layer)    
    
    def createInputNode(self):
        node = InputNode()
        self.inputLayer.addNode(node)
        self.nodeList.append(node)
        return node
    
    def createOutputNode(self):
        node = OutputNode()
        self.outputLayer.addNode(node)
        self.nodeList.append(node)
        return node
    
    def createHiddenNode(self, hiddenLayer: HiddenLayer):
        node = HiddenNode()
        hiddenLayer.addNode(node)
        self.nodeList.append(node)
        return node

    def createMultipleInputNodes(self, count : int):        
        for i in range(count):
            self.createInputNode()            

    def createMultipleOutputNodes(self, count : int):        
        for i in range(count):
            self.createOutputNode()            

    def createMultipleHiddenNodes(self, hiddenLayer : HiddenLayer, count : int):        
        for i in range(count):
            self.createHiddenNode(hiddenLayer)

    def connectInputToHidden(self):
        for fromNode in self.inputLayer.nodes:
            for toNode in self.hiddenLayerList[0].nodes:
                edge = Edge(fromNode, toNode)
                self.edgeList.append(edge)
                fromNode.addOutEdge(edge)
                toNode.addInEdge(edge)

    def connectHiddenToOutput(self):
        for fromNode in self.hiddenLayerList[0].nodes:
            for toNode in self.outputLayer.nodes:
                edge = Edge(fromNode, toNode)
                self.edgeList.append(edge)
                fromNode.addOutEdge(edge)
                toNode.addInEdge(edge)
            
    def calculateInitialWeights(self):
        for edge in self.edgeList:
            edge.weight = random.uniform(0, 1/10)

    def readDf(self, dataframe : DataFrame):
        # self.trainDf = dataframe
        self.trainDf = dataframe.sample(frac=0.8)
        self.testDf = dataframe.drop(self.trainDf.index)    
        self.trainDf.reset_index(drop=True, inplace=True)
        self.testDf.reset_index(drop=True, inplace=True)     
        # Read Max, Min value of Each column and store in a List                
        for i in range(self.trainDf.shape[1]-1):
            self.maxAttribList.append(pd.DataFrame.max(self.trainDf.iloc[:,[i]]))
            self.minAttribList.append(pd.DataFrame.min(self.trainDf.iloc[:,[i]]))

    def inputLayerFeed(self, row : DataFrame):        
        x = 0
        for node in self.inputLayer.nodes:
            normalized = float((row[x]-self.minAttribList[x])/(self.maxAttribList[x] - self.minAttribList[x]))
            node.value = normalized
            x+=1
        self.targetList = [0.2] * len(self.outputLayer.nodes)
        self.targetList[row[-1]] = 0.8        
    
    def softMax(self):
        denominator = self.softMaxDenominator()
        for node in self.outputLayer.nodes:
            node.value = math.exp(node.value)/denominator
            
    def softMaxDenominator(self) -> float:
        denominator = 0.0
        for node in self.outputLayer.nodes:
            denominator += math.exp(node.value)
        return denominator

    def mse(self):
        sumVal = 0.0
        m = len(self.outputLayer.nodes)
        for i in range(m):        
            sumVal += (self.targetList[i] - self.outputLayer.nodes[i].value) ** 2
        ans = sumVal/m
#         print("mse:",ans)
        return ans

    def deltaForOutputLayer(self):
        m = len(self.outputLayer.nodes)
        for i in range(m):
            node = self.outputLayer.nodes[i]
            deltaVal = node.value * (1-node.value) * (self.targetList[i]-node.value)
            node.delta = deltaVal
            
    def deltaForHiddenLayer(self):
        m = len(self.hiddenLayerList[0].nodes)
        for i in range(m):
            node = self.hiddenLayerList[0].nodes[i]
            sumVal = 0.0
            for edge in node.outEdgeList:
                sumVal += edge.toNode.delta * edge.weight
            deltaVal = node.value * (1-node.value) * sumVal
            node.delta = deltaVal

    def updateHiddenToOutputWeights(self):
        m = len(self.hiddenLayerList[0].nodes)
        for i in range(m):
            node = self.hiddenLayerList[0].nodes[i]
            for edge in node.outEdgeList:
                edge.weight += self.learningRate * edge.toNode.delta * node.value
    
    def updateInputToHiddenWeights(self):
        m = len(self.inputLayer.nodes)
        for i in range(m):
            node = self.inputLayer.nodes[i]
            for edge in node.outEdgeList:
                edge.weight += self.learningRate * edge.toNode.delta * node.value
    
    def singlePass(self, rowNumber):
        self.inputLayerFeed(self.trainDf.iloc[rowNumber])
        for layer in self.hiddenLayerList:
            for node in layer.nodes:
                node.value = Utility.logistic(node)
        
        for node in self.outputLayer.nodes:
            node.value = Utility.logistic(node)
        
        self.softMax()
        # self.mse()
        self.deltaForOutputLayer()
        self.deltaForHiddenLayer()
        self.updateHiddenToOutputWeights()
        self.updateInputToHiddenWeights()
        
    def trainDfTest(self):
        rightAnswerCount = 0
        for i in range(self.trainDf.shape[0]):
            self.inputLayerFeed(self.trainDf.iloc[i])
            for layer in self.hiddenLayerList:
                for node in layer.nodes:
                    node.value = Utility.logistic(node)

            for node in self.outputLayer.nodes:
                node.value = Utility.logistic(node)
            
            self.softMax()
            outputList = [node.value for node in self.outputLayer.nodes]
            outputIndex = outputList.index(max(outputList))
            targetIndex = self.targetList.index(max(self.targetList))

            print("------------------------------------------------------")
            print("outputList",outputList)
            print("self.targetList",self.targetList)
            print("Target Index : ", targetIndex)
            print("Output Index:", outputIndex)
#             print("Output Difference : ", 1-outputList[outputIndex])
            if outputIndex == targetIndex:
                rightAnswerCount+=1
        print("rightAnswerCount : ",rightAnswerCount, "/Out of : ",self.trainDf.shape[0])

    def runANN(self):
        print("------------------------------------------------")
        icount = 0
        for node in self.inputLayer.nodes:    
            for edge in node.outEdgeList:
                print(icount," input edge weight:",edge.weight)
                icount+=1
        icount = 0
        print("------------------------------------------------")
        for node in self.hiddenLayerList[0].nodes:    
            for edge in node.outEdgeList:
                print(icount," hidden edge weight:",edge.weight)
                icount+=1
        
        for k in range(20):                                                          
            for i in range(self.trainDf.shape[0]):
                self.singlePass(i)
        
        # Last Pass
#         for i in range(self.trainDf.shape[0]):
#             self.singlePass(i)
#             self.mse()

        icount = 0
        print("------------------------------------------------")
        for node in self.inputLayer.nodes:    
            for edge in node.outEdgeList:
                print(icount,"input edge weight:",edge.weight)
                icount+=1
        icount = 0
        print("------------------------------------------------")
        for node in self.hiddenLayerList[0].nodes:    
            for edge in node.outEdgeList:
                print(icount," hidden edge weight:",edge.weight)  
                icount+=1

In [2]:
# from MLPv1 import *
import numpy as np
import pandas as pd
import re

def read_file(filepath) -> list:
    dataset = []        
    with open(filepath) as fp:
        for line in fp:                              
            compiler = re.compile("\d+")
            dataList = compiler.findall(line)                        
            row = list(map(int, dataList[1:]))            
            # print(row)
            dataset.append(row)
    return dataset

def createDataFrame(dataset : list):
    df = pd.DataFrame(dataset, columns=['a1','a2','a3','a4','a5','a6','a7','a8','a9','a10','label'])    
    return df


dataset = read_file("dataset.txt")
df = createDataFrame(dataset)
numberOfInputNodes = 10
numberOfOutputNodes = 8
g = Graph()
g.createHiddenLayer()
g.createMultipleInputNodes(numberOfInputNodes)
g.createMultipleHiddenNodes(g.hiddenLayerList[0], 10)
g.createMultipleOutputNodes(numberOfOutputNodes)

print("Graph total nodes : ",len(g.nodeList))
print("Input Nodes : ", len(g.inputLayer.nodes))
print("Output Nodes : ", len(g.outputLayer.nodes))
print("Hidden Nodes : ", len(g.hiddenLayerList[0].nodes))

g.connectInputToHidden()
g.connectHiddenToOutput()
g.calculateInitialWeights()
g.readDf(df)

# g.singlePass()

# for node in g.inputLayer.nodes:    
#     print("input node value:",node.value)

# print("------------------------------------------------")
# for node in g.hiddenLayerList[0].nodes:    
#     print("hidden node value:",node.value)

# print("------------------------------------------------")
# for node in g.outputLayer.nodes:    
#     print("output node value:",node.value)

# g.updateHiddenToOutputWeights()
# g.updateInputToHiddenWeights()
# g.singlePass()
# print("------------------------------------------------")
# for node in g.inputLayer.nodes:    
#     print("input node value:",node.value)

# print("------------------------------------------------")
# for node in g.hiddenLayerList[0].nodes:    
#     print("hidden node value:",node.value)

# print("------------------------------------------------")
# for node in g.outputLayer.nodes:    
#     print("output node value:",node.value)


Graph total nodes :  28
Input Nodes :  10
Output Nodes :  8
Hidden Nodes :  10


In [3]:
g.trainDfTest()

------------------------------------------------------
outputList [0.121735368274069, 0.12442234097653682, 0.12332984428112242, 0.1281305772617595, 0.12577857908073337, 0.1242928708040164, 0.12553255073707545, 0.126777868584687]
self.targetList [0.2, 0.2, 0.2, 0.8, 0.2, 0.2, 0.2, 0.2]
Target Index :  3
Output Index: 3
------------------------------------------------------
outputList [0.12174194306490019, 0.12444490285790016, 0.12332253117223609, 0.12813263222837765, 0.12578406479809376, 0.12428063737694872, 0.12553131039560514, 0.12676197810593837]
self.targetList [0.2, 0.8, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2]
Target Index :  1
Output Index: 3
------------------------------------------------------
outputList [0.12172929156450707, 0.12444518984033538, 0.12330849075348789, 0.1281015978896591, 0.12582319781340306, 0.12425942895371969, 0.1255689688459739, 0.1267638343389138]
self.targetList [0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.8, 0.2]
Target Index :  6
Output Index: 3
-------------------------------

self.targetList [0.2, 0.8, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2]
Target Index :  1
Output Index: 3
------------------------------------------------------
outputList [0.12173452222653239, 0.12443344819157727, 0.1233120605937564, 0.12813116206249203, 0.1258031073230115, 0.12429162993490668, 0.12554627853562003, 0.12674779113210374]
self.targetList [0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.8, 0.2]
Target Index :  6
Output Index: 3
------------------------------------------------------
outputList [0.12170259764234607, 0.12444021977746737, 0.12329084050749277, 0.12814945057580532, 0.12582306027412907, 0.1242439038234511, 0.1255549649884665, 0.1267949624108417]
self.targetList [0.2, 0.2, 0.8, 0.2, 0.2, 0.2, 0.2, 0.2]
Target Index :  2
Output Index: 3
------------------------------------------------------
outputList [0.12167530413633575, 0.12439471330164903, 0.12326640129911492, 0.1281860867763618, 0.12585555558642053, 0.12426737899076316, 0.12555517133004865, 0.12679938857930603]
self.targetList [0.2, 0.2, 0

Output Index: 3
------------------------------------------------------
outputList [0.12167338182674192, 0.12439484241816123, 0.12326543929889236, 0.12818806415006667, 0.1258558349920725, 0.12426687115233546, 0.12555582854899688, 0.12679973761273294]
self.targetList [0.2, 0.2, 0.2, 0.2, 0.2, 0.8, 0.2, 0.2]
Target Index :  5
Output Index: 3
------------------------------------------------------
outputList [0.12163466364324634, 0.12440551256940874, 0.12326967762846729, 0.1282332330354914, 0.12582810469435127, 0.12426265355220618, 0.12556541959839151, 0.1268007352784373]
self.targetList [0.2, 0.2, 0.2, 0.8, 0.2, 0.2, 0.2, 0.2]
Target Index :  3
Output Index: 3
------------------------------------------------------
outputList [0.12193775513613482, 0.1244794657328838, 0.12341288969013378, 0.12790335174556353, 0.12575773537444138, 0.12433207297915465, 0.1255110996792545, 0.12666562966243355]
self.targetList [0.2, 0.2, 0.2, 0.2, 0.2, 0.8, 0.2, 0.2]
Target Index :  5
Output Index: 3
-----------

In [4]:
g.runANN()

------------------------------------------------
0  input edge weight: 0.01809540552476966
1  input edge weight: 0.027264152626628402
2  input edge weight: 0.0221576956184959
3  input edge weight: 0.08949720079278843
4  input edge weight: 0.025869497618488214
5  input edge weight: 0.006943391217592332
6  input edge weight: 0.06392400094044931
7  input edge weight: 0.09446030724669603
8  input edge weight: 0.028640374549471315
9  input edge weight: 0.08597256228401176
10  input edge weight: 0.07076133819425318
11  input edge weight: 0.0801960549367366
12  input edge weight: 0.03278364855648388
13  input edge weight: 0.055481819956196124
14  input edge weight: 0.07606073117313467
15  input edge weight: 0.06482292252475201
16  input edge weight: 0.0784242981731147
17  input edge weight: 0.09948612510881094
18  input edge weight: 0.011596192296989695
19  input edge weight: 0.010834785680595184
20  input edge weight: 0.024579690070591922
21  input edge weight: 0.02990705211275213
22  input 

In [5]:
g.trainDfTest()

------------------------------------------------------
outputList [0.12500000039286066, 0.1250000074065038, 0.12500000748958254, 0.12500000748969403, 0.12500000748882667, 0.1250000074888905, 0.12500000686895288, 0.12499995537468875]
self.targetList [0.2, 0.2, 0.2, 0.8, 0.2, 0.2, 0.2, 0.2]
Target Index :  3
Output Index: 3
------------------------------------------------------
outputList [0.1250000003606039, 0.12500000794074181, 0.1250000080324266, 0.12500000803255298, 0.12500000803157624, 0.12500000803164787, 0.12500000735385544, 0.12499995221659539]
self.targetList [0.2, 0.8, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2]
Target Index :  1
Output Index: 3
------------------------------------------------------
outputList [0.12500000038460732, 0.12500000755380639, 0.12500000763922875, 0.12500000763934424, 0.12500000763844737, 0.1250000076385132, 0.12500000700279276, 0.12499995450325986]
self.targetList [0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.8, 0.2]
Target Index :  6
Output Index: 3
---------------------------

Output Index: 3
------------------------------------------------------
outputList [0.12500000042669704, 0.12500000673463, 0.12500000680723797, 0.125000006807332, 0.1250000068065945, 0.125000006806649, 0.1250000062578627, 0.12499995935299679]
self.targetList [0.2, 0.8, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2]
Target Index :  1
Output Index: 3
------------------------------------------------------
outputList [0.12499999951172679, 0.12500001562057256, 0.1250000158593924, 0.12500001585981668, 0.12500001585672438, 0.12500001585694387, 0.12500001424501572, 0.12499990718980747]
self.targetList [0.2, 0.8, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2]
Target Index :  1
Output Index: 3
------------------------------------------------------
outputList [0.12500000038771075, 0.12500000749927628, 0.12500000758382676, 0.1250000075839408, 0.1250000075830549, 0.12500000758312005, 0.12500000695326405, 0.12499995482580628]
self.targetList [0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.8, 0.2]
Target Index :  6
Output Index: 3
------------------

------------------------------------------------------
outputList [0.12500000040801298, 0.1250000071211564, 0.12500000719973958, 0.12500000719984353, 0.12500000719903234, 0.12500000719909213, 0.1250000066095786, 0.12499995706354437]
self.targetList [0.2, 0.8, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2]
Target Index :  1
Output Index: 3
------------------------------------------------------
outputList [0.12499999996248437, 0.12500001225128562, 0.12500001242065695, 0.12500001242093162, 0.12500001241888706, 0.12500001241903375, 0.1250000112380352, 0.12499992686868534]
self.targetList [0.2, 0.2, 0.8, 0.2, 0.2, 0.2, 0.2, 0.2]
Target Index :  2
Output Index: 3
------------------------------------------------------
outputList [0.12500000021882057, 0.12500000978420428, 0.12500000990740903, 0.12500000990759266, 0.12500000990619878, 0.12500000990629986, 0.12500000902079106, 0.12499994134868368]
self.targetList [0.2, 0.2, 0.8, 0.2, 0.2, 0.2, 0.2, 0.2]
Target Index :  2
Output Index: 3
-------------------------

In [6]:
values, count = np.unique(g.trainDf['label'], return_counts=True)
print(values, count)

[0 1 2 3 4 5 6 7] [ 5  9 15 15 13 13  7  3]
